In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

In [2]:
format_dict = {
               'price':'{:.2f}','buy_price':'{:.2f}','sell_price':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

### Change kind en masse

In [3]:
kind = 'DIV'
year = 2020
quarter = 4

### Stocks that cannot be sold yet

In [4]:
sql = '''
SELECT T.name, B.date AS buy_date, qty, B.price AS buy_price, B.net AS buy_amt
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE kind = "%s"
AND status = 'Active'
ORDER BY T.name, B.date'''
sql = sql % kind
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,name,buy_date,qty,buy_price,buy_amt
0,BGC,2021-03-12,"10,000",10.20,"102,225.92"
1,BGC,2021-05-07,"10,000",9.60,"96,212.63"
2,BKI,2021-05-18,300,275.00,"82,682.73"
3,DCC,2021-01-28,"40,000",2.34,"93,807.31"
4,HREIT,2020-08-14,"12,000",8.20,"98,617.95"
5,HREIT,2020-09-01,"6,000",8.00,"48,106.32"
6,HREIT,2020-09-25,"6,000",7.90,"47,504.99"
7,HREIT,2020-09-28,100,7.85,786.73
8,HREIT,2020-10-15,"5,900",7.85,"46,417.58"
9,LH,2018-08-23,"30,000",11.50,"345,764.14"


In [5]:
buy_amt = buys.buy_amt.sum()
round(buy_amt,2)

2029394.94

### Stocks that are sold before reaching dividend date

In [16]:
sql = '''
SELECT T.name, qty, S.price AS sell_price, B.price AS buy_price, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE status = "Sold" AND kind = "%s"
AND S.dividend_id = 0
ORDER BY T.name'''
sql = sql % kind

sells_b_div = pd.read_sql(sql, conpf)
sells_b_div.sort_values(['sell_date'],ascending=[False]).style.format(format_dict)

,name,qty,sell_price,buy_price,diff,sell_amt,buy_amt,profit,sell_date,buy_date,days,pct,id,kind
17,TTLPF,"1,000",23.00,21.90,1.10,"22,949.06","21,948.50","1,000.56",2021-05-14,2020-05-25,354,4.70,715,DIV
15,TTLPF,"1,000",23.00,21.90,1.10,"22,949.06","21,948.50","1,000.56",2021-05-14,2020-05-26,353,4.71,714,DIV
13,TTLPF,"1,000",23.30,21.60,1.70,"23,248.39","21,647.84","1,600.55",2021-05-12,2020-05-28,349,7.73,711,DIV
10,TTLPF,"1,000",23.30,21.70,1.60,"23,248.39","21,748.06","1,500.33",2021-05-06,2020-06-26,314,8.02,702,DIV
11,TTLPF,200,23.30,21.70,1.60,"4,649.67","4,349.61",300.06,2021-05-06,2020-05-28,343,7.34,703,DIV
16,TTLPF,100,23.00,21.60,1.40,"2,294.91","2,164.78",130.13,2021-04-02,2020-11-13,140,15.67,682,DIV
14,TTLPF,"1,000",23.00,21.70,1.30,"22,949.06","21,748.06","1,201.00",2021-04-02,2020-09-18,196,10.28,681,DIV
12,TTLPF,200,23.00,22.10,0.90,"4,589.81","4,429.79",160.02,2021-03-12,2021-02-10,30,43.95,667,DIV
6,SIRI,"100,000",0.70,0.81,-0.11,"69,844.96","81,179.41","-11,334.45",2020-04-28,2020-03-09,50,-101.92,560,DIV
5,LPF,"3,000",18.20,17.60,0.60,"54,479.07","52,916.95","1,562.12",2020-04-24,2020-04-21,3,359.16,558,DIV


In [8]:
profit = round(sells_b_div.profit.sum(),2)
cost = round(sells_b_div.buy_amt.sum(),2)
avg_day = sells_b_div.days.mean()
pct = round(profit/cost*36500/avg_day,2)
profit,cost,pct

(16300.77, 718558.0, 6.79)

### Bind sell to dividend record process

In [9]:
sr = sells_b_div["name"]
names = sr.values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'BFIT', 'GC', 'GC', 'KGI', 'KKP', 'LPF', 'SIRI', 'TISCO', 'TKS', 'TKS', 'TTLPF', 'TTLPF', 'TTLPF', 'TTLPF', 'TTLPF', 'TTLPF', 'TTLPF', 'TTLPF', 'TTW', 'TTW'"

In [10]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, X_date, p_date
FROM dividends
WHERE name IN (%s)
ORDER BY name, year DESC, quarter DESC
"""
#AND year = %s AND quarter = %s

sql = sql % (in_p)
#sql = sql % (in_p, year, quarter)
dividends = pd.read_sql(sql, conpf)
#dividends.set_index(['name'],inplace=True)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,X_date,p_date
0,25,BFIT,2019,4,"1,000",2.2500,"2,025.00",2020-04-24,2020-04-24
1,214,GC,2020,4,"15,000",0.4000,"5,400.00",2021-04-16,2021-04-30
2,125,GC,2018,1,"10,000",0.2200,"1,980.00",2018-04-30,2018-04-30
3,21,KKP,2019,4,"1,200",2.7500,"2,970.00",2020-04-28,2020-04-28
4,60,KKP,2018,4,"8,000",3.0000,"21,600.00",2019-05-21,2019-05-21
5,194,LPF,2020,3,"7,000",0.1955,"1,231.65",2021-01-27,2021-02-15
6,11,SIRI,2019,4,"100,000",0.0600,"5,400.00",2020-05-12,2020-05-12
7,222,TISCO,2020,4,"2,000",6.3000,"11,340.00",2021-04-28,2021-05-17
8,62,TISCO,2018,4,"6,000",7.0000,"37,800.00",2019-05-17,2019-05-17
9,114,TISCO,2018,1,"4,500",5.0000,"20,250.00",2018-05-18,2018-05-18


In [11]:
df_merge = pd.merge(sells_b_div, dividends, on='name', how='inner')
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct']  = round(df_merge.div_amt / df_merge.buy_amt * 36500 / df_merge.days,2)
df_merge.style.format(format_dict)

,name,qty,sell_price,buy_price,diff,sell_amt,buy_amt,profit,sell_date,buy_date,days,pct,id_x,kind,id_y,year,quarter,number,ppu,div_amt,X_date,p_date
0,BFIT,"1,000",25.25,24.80,0.45,"25,194.07","24,854.93",339.14,2020-04-07,2020-03-24,31,95.93,544,DIV,25,2019,4,"1,000",2.2500,"2,025.00",2020-04-24,2020-04-24
1,GC,"3,100",4.90,4.50,0.40,"15,156.36","13,980.90","1,175.46",2020-03-26,2020-03-24,402,35.07,542,DIV,214,2020,4,"15,000",0.4000,"5,400.00",2021-04-16,2021-04-30
2,GC,"3,100",4.90,4.50,0.40,"15,156.36","13,980.90","1,175.46",2020-03-26,2020-03-24,-694,-7.45,542,DIV,125,2018,1,"10,000",0.2200,"1,980.00",2018-04-30,2018-04-30
3,GC,"5,000",4.90,4.50,0.40,"24,445.73","22,549.84","1,895.89",2020-03-26,2020-03-19,407,21.48,543,DIV,214,2020,4,"15,000",0.4000,"5,400.00",2021-04-16,2021-04-30
4,GC,"5,000",4.90,4.50,0.40,"24,445.73","22,549.84","1,895.89",2020-03-26,2020-03-19,-689,-4.65,543,DIV,125,2018,1,"10,000",0.2200,"1,980.00",2018-04-30,2018-04-30
5,KKP,600,42.00,37.00,5.00,"25,144.19","22,249.17","2,895.02",2020-04-10,2020-03-30,29,168.01,549,DIV,21,2019,4,"1,200",2.7500,"2,970.00",2020-04-28,2020-04-28
6,KKP,600,42.00,37.00,5.00,"25,144.19","22,249.17","2,895.02",2020-04-10,2020-03-30,-314,-112.85,549,DIV,60,2018,4,"8,000",3.0000,"21,600.00",2019-05-21,2019-05-21
7,LPF,"3,000",18.20,17.60,0.60,"54,479.07","52,916.95","1,562.12",2020-04-24,2020-04-21,300,2.83,558,DIV,194,2020,3,"7,000",0.1955,"1,231.65",2021-01-27,2021-02-15
8,SIRI,"100,000",0.70,0.81,-0.11,"69,844.96","81,179.41","-11,334.45",2020-04-28,2020-03-09,64,37.94,560,DIV,11,2019,4,"100,000",0.0600,"5,400.00",2020-05-12,2020-05-12
9,TISCO,500,81.50,68.00,13.50,"40,659.75","34,075.31","6,584.44",2020-04-16,2020-03-30,413,29.41,552,DIV,222,2020,4,"2,000",6.3000,"11,340.00",2021-04-28,2021-05-17


### dividend_id = id_y, id = id_x 

In [13]:
sql = '''
UPDATE sells
SET dividend_id = 15
WHERE id = 579'''
rp = conpf.execute(sql)
rp.rowcount

1

### End of Bind sell to dividend record process

In [14]:
cols = 'name buy_date p_date qty buy_amt div_amt days pct'.split()
df_merge[cols].style.format(format_dict)

,name,buy_date,p_date,qty,buy_amt,div_amt,days,pct
0,BFIT,2020-03-24,2020-04-24,"1,000","24,854.93","2,025.00",31,95.93
1,GC,2020-03-24,2021-04-30,"3,100","13,980.90","5,400.00",402,35.07
2,GC,2020-03-24,2018-04-30,"3,100","13,980.90","1,980.00",-694,-7.45
3,GC,2020-03-19,2021-04-30,"5,000","22,549.84","5,400.00",407,21.48
4,GC,2020-03-19,2018-04-30,"5,000","22,549.84","1,980.00",-689,-4.65
5,KKP,2020-03-30,2020-04-28,600,"22,249.17","2,970.00",29,168.01
6,KKP,2020-03-30,2019-05-21,600,"22,249.17","21,600.00",-314,-112.85
7,LPF,2020-04-21,2021-02-15,"3,000","52,916.95","1,231.65",300,2.83
8,SIRI,2020-03-09,2020-05-12,"100,000","81,179.41","5,400.00",64,37.94
9,TISCO,2020-03-30,2021-05-17,500,"34,075.31","11,340.00",413,29.41


### Percent calculation of sells after dividend payment date

In [15]:
sql = """
SELECT name, number, ppu, D.net AS div_amt, profit, profit + D.net AS ttl_amt, B.net AS buy_amt,  
p_date, S.date AS sell_date, B.date AS buy_date, DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS days,
(profit + D.net)/B.net*36500/DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS pct, year, quarter
FROM sells S 
JOIN buys B ON S.buy_id = B.id
JOIN dividends D
ON dividend_id = D.id
ORDER BY name
"""
sells_a_div = pd.read_sql(sql, conpf)
sells_a_div.sort_values(['p_date'],ascending=[False]).style.format(format_dict)

,name,number,ppu,div_amt,profit,ttl_amt,buy_amt,p_date,sell_date,buy_date,days,pct,year,quarter
5,PDG,"25,000",0.2200,"4,950.00",-431.92,"4,518.08","97,715.96",2021-05-20,2021-05-12,2021-03-12,69,24.46,2020,4
6,TIP,"3,000",1.3000,"3,510.00",349.66,"3,859.66","90,199.34",2021-05-19,2021-05-06,2021-03-10,70,22.31,2020,4
3,MBAX,"12,000",0.5600,"6,048.00","-22,633.46","-16,585.46","154,341.09",2021-05-07,2021-05-12,2021-04-12,30,-130.74,2020,4
0,DTAC,"9,000",2.1200,"17,172.00","-5,805.69","11,366.31","297,657.83",2021-04-20,2021-05-05,2021-02-04,90,15.49,2020,4
9,WHART,"10,000",0.1195,"1,075.50",492.79,"1,568.29","114,252.50",2021-03-22,2021-03-04,2021-03-03,19,26.37,2020,4
1,EGATIF,"10,000",0.2000,"2,000.00","-1,538.22",461.78,"122,270.22",2021-03-17,2021-05-13,2021-02-25,77,1.79,2020,4
4,MC,"10,000",0.3500,"3,150.00",528.23,"3,678.23","106,234.78",2021-03-11,2021-03-03,2021-02-24,15,84.25,2020,2
2,GVREIT,"18,000",0.2037,"3,299.94",918.91,"4,218.85","198,438.55",2021-03-10,2021-03-10,2021-02-15,23,33.74,2020,4
7,TTW,"20,000",0.3000,"5,400.00",-446.62,"4,953.38","101,043.31",2020-04-30,2020-06-05,2020-03-06,91,19.66,2019,4
8,TTW,"20,000",0.3000,"5,400.00",-811.44,"4,588.56","183,585.72",2020-04-30,2020-06-04,2020-03-05,91,10.03,2019,4


In [17]:
total_profit = round(sells_a_div.ttl_amt.sum(),2)
buy_cost = round(sells_a_div.buy_amt.sum(),2)
avg_day = sells_a_div.days.mean()
sell_pct = round(total_profit/buy_cost*36500/avg_day,2)
total_profit,buy_cost,sell_pct

(13085.74, 1181110.27, 8.23)

In [7]:
sql = """
SELECT name, COUNT(*)
FROM dividends
GROUP BY name
ORDER BY COUNT(*) DESC
"""
df = pd.read_sql(sql, conpf)
df.head(20)

,name,COUNT(*)
0,IVL,12
1,JASIF,12
2,DIF,11
3,CPNREIT,9
4,LH,8
5,ORI,8
6,MCS,8
7,HREIT,7
8,FSMART,7
9,AMATA,7
